In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('D:\\Dropbox\\stempy')
sys.path.append('D:\\Dropbox\\stemplot')
    
from stemplot import *

from stempy.io import *
from stempy.clustering import *
from stempy.plot import *
from stempy.denoise import *
from stempy.utils import *
from stempy.feature import *
from stempy.spatial import *
from stempy.manifold import *

In [2]:
dp = r'D:\Dropbox\data\Leyi\Dopant Valency Project\VWSe2_total 10 datasets\VWSe2\VWSe2_20210930_E2'+'\\'

# must change this
num = 7

file_name1 = dp + 'ADF1_{}.dm4'.format(num)
file_name2 = dp + 'ADF2_{}.dm4'.format(num)


# load image and normalize
img1 = load_image(file_name1)
img1 = normalize_image(img1, 0, 1)

img2 = load_image(file_name2)
img2 = normalize_image(img2, 0, 1)

In [3]:
img1 = remove_bg(img1, 7)
img2 = remove_bg(img2, 7)

img1 = normalize_image(img1, 0, 1)
img2 = normalize_image(img2, 0, 1)
plot_compare([img1, img2])

In [4]:
get_window_size(img1), get_window_size(img2)

(16, 16)

## denoising

In [5]:
n_components = 32
patch_size = 32
extraction_step = 2
imgf = denoise_svd(img2, n_components, patch_size, extraction_step) 

Extracting reference patches...
done in 0.74s.
Singular value decomposition...
done in 9.20s.
Reconstructing...
done in 4.41s.


In [6]:
imshow(imgf)

In [7]:
threshold = 0.2
pts = local_max(imgf, min_distance=2, threshold=threshold)

In [8]:
size = 33
kp1 = KeyPoints(pts, img1, size)
ps1 = kp1.extract_patches(size) 

kp2 = KeyPoints(pts, img2, size)
ps2 = kp2.extract_patches(size) 

In [9]:
imshow(ps2)

In [10]:
zps = ZPs(n_max=12, size=ps1.shape[1])
zps.fit(ps1)
X1 = zps.moments
zps.fit(ps2)
X2 = zps.moments

X = np.hstack([X1, X2])

In [11]:
plot_pca(X, 2)

In [12]:
lbs = gmm_lbs(X, 2)

In [13]:
plot_pca(X, 2, lbs=lbs)

In [14]:
X11 = X[lbs==0]
X22 = X[lbs==1]

In [15]:
plot_pca(X11, 2, s=3)

In [16]:
lbs_selected = 0
if lbs_selected == 0:
    fg = ForceGraph8(X = X11,
    n_neighbors=10,
    init_mode = 'pca',
    num_iterations = 100,
    num_negative_samples=5,
    force_params1=(0, 2, 1, 1),
    force_params2=(2, 5, 5, 1),
    divide=0.5)
    
    from time import time
    t0 = time()
    xy = fg.fit_transform(X11)
else:
    fg = ForceGraph8(X = X22,
    n_neighbors=10,
    init_mode = 'pca',
    num_iterations = 100,
    num_negative_samples=5,
    force_params1=(0, 2, 1, 1),
    force_params2=(2, 5, 5, 1),
    divide=0.5)
    
    from time import time
    t0 = time()
    xy = fg.fit_transform(X22)

Construct graph from data...
Initialize 2-d embedding using PCA layout...


In [17]:
l= interactive_clusters(xy, img2, kp1.pts[lbs==lbs_selected], ps2[lbs==lbs_selected], clip=False, s=1) 

One cluster has been selected.


In [18]:
len(l.ind)

80

In [445]:
from sklearn.manifold import TSNE
Xt = TSNE(n_components=2, n_iter=2000).fit_transform(X22)

In [531]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 7.2))
ax.scatter(Xt[:, 0], Xt[:, 1], s=3)

In [532]:
l= interactive_clusters(Xt, img, kp1.pts[lbs==1], ps2[lbs==1], clip=False, s=1) 

In [450]:
motif1 = ps1[lbs==1][l.ind].mean(axis=0)
motif2 = ps2[lbs==1][l.ind].mean(axis=0)
imshow(motif1)

In [518]:
from skimage.util import random_noise

shape = (X22.shape[0], size, size)
ps3 = np.broadcast_to(motif1, shape)
ps4 = np.broadcast_to(motif2, shape)
ps3 = random_noise(ps3, var=0.01)
ps4 = random_noise(ps4, var=0.01)


zps.fit(ps3)
X3 = zps.moments
zps.fit(ps3)
X4 = zps.moments

X34 = np.hstack([X3, X4])
Xnew = np.vstack([X22, X34])

In [528]:
lbsnew = kmeans_lbs(Xnew, 2)

C:\Users\Ted\anaconda3\envs\jiadong\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [529]:
plot_pca(Xnew, lbs=lbsnew)

In [530]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 7.2))
ax.scatter(Xtnew[:, 0], Xtnew[:, 1], s=3, color=colors_from_lbs(lbsnew))

In [520]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 7.2))
ax.scatter(Xnew[:, 0], Xnew[:, 1], s=3, color=colors_from_lbs(lbsnew))

In [486]:
Xtnew = TSNE(n_components=2, n_iter=2000).fit_transform(Xnew)

In [509]:
lbsnew = [0]*(X22.shape[0])+[1]*(X22.shape[0])

In [521]:
plot_pca(Xnew, lbs=lbsnew)

In [523]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 7.2))
ax.scatter(Xtnew[:, 0], Xtnew[:, 1], s=3, color=colors_from_lbs(lbsnew))

In [292]:
motif1 = X[lbs==1].mean(axis=0)

In [293]:
from skimage.metrics import structural_similarity
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
kk = np.array([1-cosine(e, motif1) for e in X[lbs==1]])

In [294]:
plt.hist(kk, bins=200)

(array([  1.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   1.,   0.,   1.,   0.,   0.,   1.,   0.,   0.,   1.,   1.,
          1.,   0.,   0.,   0.,   1.,   1.,   0.,   2.,   0.,   0.,   2.,
          0.,   2.,   1.,   2.,   3.,   1.,   1.,   2.,   3.,   4.,   2.,
          1.,   3.,   1.,   2.,   2.,   3.,   3.,   1.,   1.,   3.,   5.,
          6.,   6.,   5.,   9.,   7., 

In [128]:
hh = ps_[lbs==0][:, 14-1:14+2, 14-1:14+2].mean(axis=(1,2))

In [129]:
plt.hist(hh, bins=100)

(array([  1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          3.,   0.,   2.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   1.,
          0.,   0.,   1.,   2.,   1.,   2.,   1.,   4.,   2.,  10.,  13.,
         14.,  22.,  25.,  57.,  33.,  76.,  82., 109., 101., 134., 136.,
        217., 209., 239., 267., 285., 309., 298., 305., 328., 320., 267.,
        291., 245., 235., 184., 194., 132., 114.,  94.,  79.,  63.,  42.,
         45.,  35.,  29.,  29.,  14.,  12.,   4.,  10.,   7.,   4.,   0.,
          2.,   2.,   3.,   5.,   1.,   0.,   1.,   1.,   1.,   0.,   0.,
          1.]),
 array([0.04777327, 0.05260071, 0.05742816, 0.06225561, 0.06708306,
        0.07191051, 0.07673796, 0.0815654 , 0.08639285, 0.0912203 ,
        0.09604775, 0.1008752 , 0.10570264, 0.11053009, 0.11535754,
        0.12018499, 0.12501244, 0.12983989, 0.13466733, 0.13949478,
        0.14432223, 0.14914968, 0.15397713, 0.

In [125]:
fg = ForceGraph8(X = X,
n_neighbors=10,
init_mode = 'pca',
num_iterations = 100,
num_negative_samples=5,
force_params1=(0, 2, 1, 1),
force_params2=(2, 5, 5, 1),
divide=0.5)

from time import time
t0 = time()
xy = fg.fit_transform(X)

Construct graph from data...
Initialize 2-d embedding using PCA layout...


In [336]:
l= interactive_clusters(xy, img, kp.pts, ps, clip=False) 

In [15]:
l.ind

array([  24,   95,  133,  157,  194,  250,  310,  447,  597,  612,  626,
        671,  690,  851,  877,  882,  913, 1041, 1109, 1112, 1120, 1351,
       1365, 1407, 1437, 1470, 1534, 1539, 1697, 1716, 1793, 1822, 2018,
       2064, 2167, 2213, 2386, 2407, 2416, 2445, 2499, 2560, 2719, 2829,
       2915, 2920, 2930, 2942, 2944, 2957, 3025, 3041, 3100, 3206, 3229,
       3243, 3359, 3367, 3375, 3393, 3523, 3531, 3562, 3594, 3676, 3703,
       3892, 4037, 4263, 4378, 4391, 4440, 4449, 4478, 4485, 4576, 4586,
       4590, 4642, 4665, 4727, 4740, 4761, 4772, 4809, 4854, 4863, 4897,
       4910, 4925, 4975, 4982, 5030, 5055, 5077, 5398, 5413, 5415, 5580,
       5628, 5690, 5699, 5882, 5883, 5891, 5970, 5972, 6031, 6111, 6154,
       6167, 6248, 6264, 6349, 6387, 6428, 6432, 6438, 6541, 6554, 6558,
       6562, 6565, 6651, 6674, 6693, 6934, 7006, 7061, 7142, 7144, 7194,
       7334, 7348, 7355, 7421, 7452, 7603, 7682, 7697, 7745, 7846],
      dtype=int64)

In [16]:
kp.pts

array([[ 48,   8],
       [ 78,   8],
       [121,   8],
       ...,
       [452, 504],
       [461, 504],
       [482, 504]], dtype=int64)

In [17]:
np.save('pts_Ti_25.npy', kp.pts)

In [18]:
np.save('ind_Ti_25.npy', l.ind)